<a href="https://colab.research.google.com/github/ahmedbenissa/uuuuu/blob/main/final_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install git+https://github.com/m-bain/whisperx.git

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-9ua0t28j
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-9ua0t28j
  Resolved https://github.com/m-bain/whisperx.git to commit a2af56983826931f9e67f413913080980e36528d
  Preparing metadata (setup.py) ... done


In [3]:
!pip install git+https://github.com/m-bain/whisperx.git --upgrade

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-viz092uh
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-viz092uh
  Resolved https://github.com/m-bain/whisperx.git to commit a2af56983826931f9e67f413913080980e36528d
  Preparing metadata (setup.py) ... done


In [14]:
import whisperx
import gc

device = "cpu"
audio_file = r"download_1.wav"
batch_size = 16 # reduce if low on GPU mem
compute_type = "float32" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("medium", device, compute_type=compute_type)


No language specified, language will be first be detected for each audio file (increases inference time).


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.0.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.0+cu118. Bad things might happen unless you revert torch to 1.x.


In [ ]:
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)

In [ ]:
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment


Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_voxpopuli_base_10k_asr_de.pt" to /root/.cache/torch/hub/checkpoints/wav2vec2_voxpopuli_base_10k_asr_de.pt
100%|██████████| 360M/360M [00:02<00:00, 127MB/s] 


[{'start': 1.717, 'end': 8.364, 'text': ' Hallo, schönen guten Tag, mein Name ist Marco Hanzer vom Firma Sorg-Borbein.', 'words': [{'word': 'Hallo,', 'start': 1.717, 'end': 3.399, 'score': 0.8}, {'word': 'schönen', 'start': 3.419, 'end': 3.639, 'score': 0.265}, {'word': 'guten', 'start': 3.659, 'end': 3.9, 'score': 0.352}, {'word': 'Tag,', 'start': 3.92, 'end': 4.16, 'score': 0.828}, {'word': 'mein', 'start': 4.18, 'end': 4.3, 'score': 0.234}, {'word': 'Name', 'start': 4.32, 'end': 4.56, 'score': 0.652}, {'word': 'ist', 'start': 4.62, 'end': 4.861, 'score': 0.78}, {'word': 'Marco', 'start': 4.901, 'end': 5.221, 'score': 0.829}, {'word': 'Hanzer', 'start': 5.261, 'end': 5.641, 'score': 0.558}, {'word': 'vom', 'start': 5.682, 'end': 5.902, 'score': 0.691}, {'word': 'Firma', 'start': 5.922, 'end': 6.302, 'score': 0.615}, {'word': 'Sorg-Borbein.', 'start': 6.342, 'end': 8.324, 'score': 0.603}]}, {'start': 8.364, 'end': 10.226, 'text': 'Grüß Sie wohl.', 'words': [{'word': 'Grüß', 'start': 8

In [1]:
!pip install pydub

In [19]:
from pydub import AudioSegment

def separate_stereo_channels(audio_file, left_output, right_output):
    # Load audio file
    audio = AudioSegment.from_wav(audio_file)  # replace from_wav with from_mp3 if your source is an MP3

    # Split stereo audio into left and right channels
    channels = audio.split_to_mono()

    # There should be 2 channels (left and right) for a stereo audio
    if len(channels) != 2:
        raise ValueError("The provided audio file doesn't seem to be stereo.")

    left_channel = channels[1]
    right_channel = channels[0]

    # Export left and right channels
    left_channel.export(left_output, format="wav")  # replace format with "mp3" if you want to export as MP3
    right_channel.export(right_output, format="wav")
    t = 20* 60 * 1000
    duration_in_milliseconds = len(audio_file)
    duration_in_minutes = duration_in_milliseconds / (1000 * 60)
    if(duration_in_minutes>10):
     agent_aud = right_channel[:t]
     cust_aud =  left_channel[:t]
     agent_aud.export("canal_right_1.wav", format="mp3")
     cust_aud.export("canal_left_1.wav", format="mp3")


# Example Usage:
audio_file_path = r"download.wav"
separate_stereo_channels(audio_file_path, 'left_channel.wav', 'right_channel.wav')

In [21]:
!whisperx --model medium --language fr "/content/canal_left_1.wav" --compute_type float32


2023-11-07 17:05:12.428465: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 17:05:12.428531: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 17:05:12.428565: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 17:05:15.194047: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.

In [22]:
!whisperx --model medium --language fr "/content/canal_right_1.wav" --compute_type float32

2023-11-07 17:34:59.383056: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 17:34:59.383123: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 17:34:59.383168: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 17:35:02.940880: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.

In [28]:
import json

# Read the JSON data from files
with open('canal_left_1.json', 'r') as f:
    left_channel = json.load(f)
    # Add a channel_id for each segment in left channel
    for segment in left_channel["segments"]:
        segment['channel_id'] = 'customer'

with open('canal_right_1.json', 'r') as f:
    right_channel = json.load(f)
    # Add a channel_id for each segment in right channel
    for segment in right_channel["segments"]:
        segment['channel_id'] = 'agent'

# Merge the two channel segments based on their start attribute
combined_segments = sorted(left_channel["segments"] + right_channel["segments"], key=lambda x: x["start"])

# Create the combined JSON structure
combined_channel = {
    "segments": combined_segments
}

# Write the combined data to a new JSON file
with open('combined_channel.json', 'w') as f:
    json.dump(combined_channel, f, indent=4)


In [29]:
import json

# Load the combined data
with open('combined_channel.json', 'r') as f:
    combined_channel = json.load(f)

# Begin the VTT content with the header
vtt_content = "WEBVTT\n\n"

# Convert each segment into VTT format
for segment in combined_channel["segments"]:
    start_time = "{:02}:{:02}.{:03}".format(int(segment["start"] // 60), int(segment["start"] % 60), int((segment["start"] % 1) * 1000))
    end_time = "{:02}:{:02}.{:03}".format(int(segment["end"] // 60), int(segment["end"] % 60), int((segment["end"] % 1) * 1000))

    vtt_content += "{} --> {}\n".format(start_time, end_time)
    vtt_content += "{} ({})\n\n".format(segment["channel_id"],segment["text"])

# Write the VTT content to a file
with open('combined_transcription.vtt', 'w', encoding="utf-8") as f:
    f.write(vtt_content)

print("VTT file created as combined_transcription.vtt")


VTT file created as combined_transcription.vtt
